# Tutustuminen transformer-arkkitehtuuriin
## Tavoitteet
Tavoitteena on luoda malli, jolla voimme kääntää tekstiä englannista ranskaksi käyttämällä transformereita.
## Datan kuvaus
Datasetti pitää sisällään yli 100 000 elokuva-arvostelua, jotka on jaettu positiivisiksi ja negatiivisiksi arvosteluiksi. Data on tekstimuodossa, ja se täytyy jakaa tekstipareihin tätä tarkoitusta varten.

Otamme `GPU` koulutuksen käyttöön, jos laite sitä tukee.

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


Luemme datasetin läpi rivi kerrallaan. Jokaiselta riviltä otamme englanninkielisen ja ranskankielisen sanaparin ja lisäämme sen `text_pairs`-muuttujaan.

In [3]:
from keras.models import Sequential
from keras.layers import TextVectorization, Input, Embedding, LSTM, Dense

text_file = "fra-eng/fra.txt" 
with open(text_file, encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = [] 
for line in lines:                              
    english, french, license = line.split("\t")
    french = "[start] " + french + " [end]"
    text_pairs.append((english, french))

In [4]:
print(text_pairs[0:5])

[('Go.', '[start] Va ! [end]'), ('Go.', '[start] Marche. [end]'), ('Go.', '[start] En route ! [end]'), ('Go.', '[start] Bouge ! [end]'), ('Hi.', '[start] Salut ! [end]')]


## Datan Esikäsittely
Hyvän koulutuksen saavuttamisen kannalta on nyt hyvä sekoittaa datasetti. Asetamme `random.seed(10)`, koska haluamme datan menevän aina samalla tavalla sekaisin, jotta voimme kouluttaa mallia aina samanlaisella datalla.
Samalla jaamme sekoitetun datan `train`, `test` ja `val` parisetteihin.

In [5]:
import random
random.seed(10)
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

Malli käsittelee ja vektoroi englannin- ja ranskankieliset tekstit valmisteluvaiheessa käännös- tai tekstimallin koulutusta varten. Sanat ovet rajoitettu käyttämäkllä vain `15000` sanaa.

In [6]:
import tensorflow as tf 
from tensorflow.keras.layers import TextVectorization
import string
import re
  
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
 
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20                                    
 
source_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_french_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_french_texts) 

Seuraavassa koodisolussa määritämme kaksi funktiota, joiden avulla voimme muuttaa olemassaolevat sanaparit sanakirjaksi, jota voimme käyttää mallin koulutukseen.

In [7]:
batch_size = 64 
  
def format_dataset(eng, fra):
    eng = source_vectorization(eng)
    fra = target_vectorization(fra)
    return ({
        "english": eng,
        "french": fra[:, :-1],                                
    }, fra[:, 1:])                                             
 
def make_dataset(pairs):
    eng_texts, fra_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    fra_texts = list(fra_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fra_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()          
 
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [8]:
for inputs, targets in train_ds.take(1):
     print(f"inputs['english'].shape: {inputs['english'].shape}")
     print(f"inputs['french'].shape: {inputs['french'].shape}")
     print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['french'].shape: (64, 20)
targets.shape: (64, 20)


## Mallinnus
Koodi luo kaksisuuntaisen GRU-pohjaisen koodauskerroksen, joka muuntaa englanninkielisen tekstin numeerisesta sekvenssistä tiivistettyyn piilotilaan, jota voidaan käyttää esimerkiksi käännösmallissa.

In [9]:
import keras
from keras import layers

embed_dim = 256 
latent_dim = 256  
 
source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)  

Rakennamme seuraavassa koodisolussa lopullisen mallin. Malli koostuu `input`-kerroksesta, joka ottaa sisälleen sanavektorin. `Embedding`-kerroksesta, joka yhdistyy `GRU`-kerrokseen. `decoder_gru`-kerrokseen annetaan sisälle aikaisemmassa koodisolussa määritetty `encoded_source` mallinosa. Mallissa on sitten yksi `Dropout`-kerros, jonka jälkeen viimeisenä kerroksena on `Dense`-kerros softmax-aktivaatiolla.
Lopulta malli luodaan yhdistämällä `source`- ja `past_target`-kerrokset listaan ja antamalla `target_next_step` mallille y:n arvoksi.

In [10]:
past_target = Input(shape=(None,), dtype="int64", name="french")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model([source, past_target], target_next_step)

Seuraava koodisolu määrittelee mallille callback-funktion, joka tallentaa epochin välein aina parhaan mallin `val_loss`-arvon perusteella.
Annamme mallille myös optimisoijana rmsprop:in ja käytämme loss-funktiona `sparse_categorical_crossentropy`:a.

In [11]:
from keras.callbacks import ModelCheckpoint
callbacks = [
    ModelCheckpoint(filepath="seq2seqrnn.keras", save_best_only=True, monitor="val_loss"),
]
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

Seuraavassa koodisolussa koulutamme mallin 30:llä epochilla.

In [12]:
model.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

Epoch 1/30
2428/2428 [==============================] - 100s 38ms/step - loss: 1.4966 - accuracy: 0.4209 - val_loss: 1.2160 - val_accuracy: 0.5039
Epoch 2/30
2428/2428 [==============================] - 93s 38ms/step - loss: 1.2276 - accuracy: 0.5170 - val_loss: 1.0709 - val_accuracy: 0.5652
Epoch 3/30
2428/2428 [==============================] - 87s 36ms/step - loss: 1.1157 - accuracy: 0.5611 - val_loss: 1.0279 - val_accuracy: 0.5981
Epoch 4/30
2428/2428 [==============================] - 86s 36ms/step - loss: 1.0903 - accuracy: 0.5880 - val_loss: 1.0209 - val_accuracy: 0.6131
Epoch 5/30
2428/2428 [==============================] - 87s 36ms/step - loss: 1.0762 - accuracy: 0.6048 - val_loss: 1.0150 - val_accuracy: 0.6241
Epoch 6/30
2428/2428 [==============================] - 87s 36ms/step - loss: 1.0650 - accuracy: 0.6169 - val_loss: 1.0105 - val_accuracy: 0.6304
Epoch 7/30
2428/2428 [==============================] - 86s 35ms/step - loss: 1.0548 - accuracy: 0.6260 - val_loss: 1.0066 

KeyboardInterrupt: 

In [20]:
loaded_model = keras.models.load_model("seq2seqrnn.keras")

Nyt voimme tutkia mallin tuottamia käännöksiä.

In [21]:
import numpy as np
fra_vocab = target_vectorization.get_vocabulary()                          
fra_index_lookup = dict(zip(range(len(fra_vocab)), fra_vocab))             
max_decoded_sentence_length = 20
 
def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"                                           
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(                      
            [tokenized_input_sentence, tokenized_target_sentence], verbose=0)         
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])   
        sampled_token = fra_index_lookup[sampled_token_index]              
        decoded_sentence += " " + sampled_token                            
        if sampled_token == "[end]":                                       
            break
    return decoded_sentence
  
test_eng_texts = [pair[0] for pair in test_pairs] 
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 7: unexpected end of data

## Arviointi
Suurin osa mallin tuottamista käännöksistä on tarkkoja ja selkeitä lukuunottamatta puuttuvia sanoja ja pieniä kielioppivirheitä. Lauseet ovat sellaisia, että ranskaa puhuva henkilö saisi niistä kuitenkin helposti selvää. Poikkeuksena on muutama lause, jotka eivät tarkoita yhtään mitään. Mitä pidempiä englanninkielisiä lauseita yritämme kääntää, sitä huonompia käännökset tuntuvat olevan. Rajoitetun sanamäärän `15000` vuoksi usea sana näyttäisi olevan `[UNK]`.
Käännösten arvioijana toimi ranskaa äidinkielenä puhuva ryhmän jäsen.

## Käyttöönotto
Mallia voisi tietenkin hyöydyntää omana kääntösovelluksena, mutta mallin tuottamat käännökset voitaisiin implementoida suoraan johonkin tuotteeseen, jota halutaan tarjota useilla kielillä. Esimerkiksi jonkun videonpelin käännöksen voisi hoitaa samanlaisella mallilla, kunhan malli olisi koulutettu sopivalla datalla.